In [25]:
import numpy as np
import pandas as pd
from collections import defaultdict

path = '/data/mint/DPM_Dataset/Generated_Dataset/params_recreate_from_v12/train/ffhq-train-light-anno.txt'

def read_params(path):
    params = pd.read_csv(path, header=None, sep=" ", index_col=False, lineterminator='\n')
    params.rename(columns={0:'img_name'}, inplace=True)
    params = params.set_index('img_name').T.to_dict('list')
    return params

def swap_key(params):
    params_s = defaultdict(dict)
    for params_name, v in params.items():
        for img_name, params_value in v.items():
            params_s[img_name][params_name] = np.array(params_value).astype(np.float64)

    return params_s

out = swap_key({'light':read_params(path)})

In [26]:
import tqdm
json_out = {'pair':{}}
count = 0
for k in out.keys():
    if '_' in k:
        tmp = k.split('_')
        json_out['pair'][f'pair{count}'] = {
            'src':tmp[0] + '.png',
            'dst':tmp[1],
        }
        count += 1
        
print("[#] Total keys(): ", len(out.keys()))
print("[#] Total pairs: ", count)
        

[#] Total keys():  120000
[#] Total pairs:  60000


In [27]:
# Sorting by making pair0 is {src=0.png, dst=xxx.png}
json_out_sorting = {'pair':{}}
for k in json_out['pair'].keys():
    src = json_out['pair'][k]['src']
    dst = json_out['pair'][k]['dst']
    
    json_out_sorting['pair'][f'pair{src.split(".")[0]}'] = {'src':src.replace('.png', '.jpg'),
                                                            'dst':dst.replace('.png', '.jpg')}

print("[#] After sorted: ", len(json_out_sorting['pair'].keys()))

json_final_out = {'pair':{}}
for i in range(len(json_out_sorting['pair'].keys())):
    json_final_out['pair'][f'pair{i}'] = json_out_sorting['pair'][f'pair{i}']

[#] After sorted:  60000


In [28]:
import json
json.dump(json_final_out, open('gen_data_retrieved.json', 'w'), indent=4)

In [29]:
source_sample_file = '/home/mint/Dev/DiFaReli/difareli-faster/dataset_generation/sampler/sampling_generated_dataset/generated_dataset_1_seed=47.json'
with open(source_sample_file, 'r') as f:
    source_sample = json.load(f)

# Check if the source_sample is the same as the json_final_out
same = 0
for k in source_sample['pair'].keys():
    if source_sample['pair'][k] != json_final_out['pair'][k]:
        print("[!] Not the same")
        break
    else:
        same += 1
print("[#] Same: ", same)

[#] Same:  60000
